# Part2: Rating Prediction

In [5]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

# 创建SparkSession
spark = SparkSession.builder.appName('movie_lens').\
  config('spark.driver.memory', '8g') \
  .config('spark.executor.memory', '8g') \
  .getOrCreate()

# 加载数据
ratings_df = spark.read.csv("dataset/ratings.csv", inferSchema=True, header=True)
#划分数据集
(training, test) = ratings_df.randomSplit([0.7, 0.3])
# # 创建ALS模型
# als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
#           coldStartStrategy="drop")
# # 训练模型
# model_als = als.fit(training)

# # 对测试集进行预测
# predictions = model_als.transform(test)

# # 评估模型
# evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
#                                 predictionCol="prediction")
# rmse = evaluator.evaluate(predictions)
# print("Root-mean-square error = " + str(rmse))

23/11/26 08:30:04 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
# Generate top 3 recommendations for all users with the ALSModel
recommendations = model_als.recommendForAllUsers(3)
recommendations.show(truncate=False)

In [6]:

# Read additional data
tags_df = spark.read.csv("dataset/tags.csv", header=True, inferSchema=True)
movies_df = spark.read.csv("dataset/movies.csv", header=True, inferSchema=True)
genome_scores_df = spark.read.csv("dataset/genome-scores.csv", header=True, inferSchema=True)
genome_tags_df = spark.read.csv("dataset/genome-tags.csv", header=True, inferSchema=True)



In [7]:
ratings_df.printSchema()
ratings_df.show(5)
ratings_df.count()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
+------+-------+------+----------+
only showing top 5 rows



25000095

In [8]:
tags_df.printSchema()
tags_df.show(5)
tags_df.count()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- timestamp: string (nullable = true)

+------+-------+----------------+----------+
|userId|movieId|             tag| timestamp|
+------+-------+----------------+----------+
|     3|    260|         classic|1439472355|
|     3|    260|          sci-fi|1439472256|
|     4|   1732|     dark comedy|1573943598|
|     4|   1732|  great dialogue|1573943604|
|     4|   7569|so bad it's good|1573943455|
+------+-------+----------------+----------+
only showing top 5 rows



1093360

In [9]:
movies_df.printSchema()
movies_df.show(5)

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [10]:
genome_scores_df.printSchema()
genome_scores_df.show(5)
genome_scores_df.count()

root
 |-- movieId: integer (nullable = true)
 |-- tagId: integer (nullable = true)
 |-- relevance: double (nullable = true)

+-------+-----+--------------------+
|movieId|tagId|           relevance|
+-------+-----+--------------------+
|      1|    1|0.028749999999999998|
|      1|    2|0.023749999999999993|
|      1|    3|              0.0625|
|      1|    4| 0.07574999999999998|
|      1|    5|             0.14075|
+-------+-----+--------------------+
only showing top 5 rows



15584448

In [11]:
genome_tags_df.printSchema()
genome_tags_df.show(5)

root
 |-- tagId: integer (nullable = true)
 |-- tag: string (nullable = true)

+-----+------------+
|tagId|         tag|
+-----+------------+
|    1|         007|
|    2|007 (series)|
|    3|18th century|
|    4|       1920s|
|    5|       1930s|
+-----+------------+
only showing top 5 rows



In [ ]:
# Join DataFrames
# complete_data = movies_df.join(tags_df, "movieId") \
#                         .join(genome_scores_df, "movieId") \
#                         .join(genome_tags_df, "tagId") \
#                         .join(ratings_df, ["userId", "movieId"])

# genome_data=genome_scores_df.join(genome_tags_df,"tagId")

# complete_data=ratings_df.join(movies_df,"movieId")\
# .join(tags_df,"movieId")\
# .join(genome_data,["movieId","tag"])

# complete_data.printSchema()
# # complete_data.show(5)
# complete_data.count()

In [4]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoder, VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import os
import pickle
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.ml import Pipeline

# spark = SparkSession.builder.appName("PrepareData").getOrCreate()
# 创建SparkSession
spark = SparkSession.builder.appName('movie_lens').\
  config('spark.driver.memory', '8g') \
  .config('spark.executor.memory', '8g') \
  .getOrCreate()


# Read movie data
data_path = "dataset/"
df_movie = spark.read.csv(
    "dataset/movies.csv", header=True,inferSchema=True)

# # Extract movie year
# def parse_year(x):
#     year = x.strip()[-5:-1]
#     if year.startswith("19"):
#         return year
#     return "unknown"

# parse_year_udf = udf(parse_year, IntegerType())
# df_movie = df_movie.withColumn("movie_year", parse_year_udf(df_movie['title'])).drop('title')

df_movie=df_movie.drop('title')

# One-hot encoding genres
genres = df_movie.select("genres").distinct().rdd.flatMap(lambda x: x).collect()
genres_unique = list(set([i for sublist in genres for i in sublist.split("|")]))

for genres_name in genres_unique:
    col = "genres_" + genres_name
    df_movie = df_movie.withColumn(col, (df_movie.genres.contains(genres_name)).cast(IntegerType()))

df_movie = df_movie.drop('genres')
genres_col_names = ["genres_" + x for x in genres_unique]

# Read rating data

df_rating = spark.read.csv(
    "dataset/ratings.csv",
    header=True,
    inferSchema=True
)
df_rating = df_rating.withColumn('timestamp', (df_rating['timestamp'] / (365 * 24 * 3600)).cast(IntegerType()))

# Data merge
df = df_rating.join(df_movie, 'movieId')



df_X = df.select(["userId", "movieId"] + genres_col_names + ["timestamp"])
df_y = df.select(["rating"])

print("Data read completed")

df.printSchema()
df.show(200)


Data read completed
root
 |-- movieId: integer (nullable = true)
 |-- userId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)
 |-- genres_Mystery: integer (nullable = true)
 |-- genres_Horror: integer (nullable = true)
 |-- genres_Western: integer (nullable = true)
 |-- genres_Documentary: integer (nullable = true)
 |-- genres_Animation: integer (nullable = true)
 |-- genres_Drama: integer (nullable = true)
 |-- genres_Sci-Fi: integer (nullable = true)
 |-- genres_IMAX: integer (nullable = true)
 |-- genres_Action: integer (nullable = true)
 |-- genres_Romance: integer (nullable = true)
 |-- genres_(no genres listed): integer (nullable = true)
 |-- genres_Comedy: integer (nullable = true)
 |-- genres_War: integer (nullable = true)
 |-- genres_Film-Noir: integer (nullable = true)
 |-- genres_Children: integer (nullable = true)
 |-- genres_Thriller: integer (nullable = true)
 |-- genres_Adventure: integer (nullable = true)
 |-- ge

In [5]:
assembler = VectorAssembler(
    inputCols=df_X.columns,
    outputCol="features"
)

df = assembler.transform(df)

# 选择特征和目标列
df = df.select("features", "rating")

# 划分数据集为训练集和测试集
(training_data, test_data) = df.randomSplit([0.7, 0.3])

rf = RandomForestRegressor(featuresCol="features", labelCol="rating")

# 训练模型
model_rf = rf.fit(training_data)

# Predicting on test data
predictions_rf = model_rf.transform(test_data)


# Evaluate the model
evaluator = RegressionEvaluator(labelCol="rating", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions_rf)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

23/11/26 09:19:37 WARN MemoryStore: Not enough space to cache rdd_130_11 in memory! (computed 230.4 MiB so far)
23/11/26 09:19:37 WARN BlockManager: Persisting block rdd_130_11 to disk instead.
23/11/26 09:19:37 WARN MemoryStore: Not enough space to cache rdd_130_14 in memory! (computed 230.4 MiB so far)
23/11/26 09:19:37 WARN BlockManager: Persisting block rdd_130_14 to disk instead.
23/11/26 09:19:39 WARN MemoryStore: Not enough space to cache rdd_130_13 in memory! (computed 230.4 MiB so far)
23/11/26 09:19:39 WARN BlockManager: Persisting block rdd_130_13 to disk instead.
23/11/26 09:19:40 WARN MemoryStore: Not enough space to cache rdd_130_6 in memory! (computed 230.4 MiB so far)
23/11/26 09:19:40 WARN BlockManager: Persisting block rdd_130_6 to disk instead.
23/11/26 09:19:40 WARN MemoryStore: Not enough space to cache rdd_130_8 in memory! (computed 153.6 MiB so far)
23/11/26 09:19:40 WARN BlockManager: Persisting block rdd_130_8 to disk instead.
23/11/26 09:19:40 WARN MemoryStore

Root Mean Squared Error (RMSE) on test data = 1.03488


In [10]:
# model_rf.save("model_rf_1")

from pyspark.ml.regression import RandomForestRegressionModel
loaded_model = RandomForestRegressionModel.load("model_rf_1")

predictions_1 = loaded_model.transform(test_data)
rmse = evaluator.evaluate(predictions_1)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 1.03488
